# Silver Layer Transformation - Session Info

## Objective
Transform the `bronze.session_info` table into a clean Silver table with standardized column formats.

## Steps
1. **Read** data from Bronze table `bronze.session_info`.
2. **Select & Rename** columns:
   - `Round`, `Year`, `Circuit`, `SessionName`, `EventName`
   - `StartDate`, `EndDate` (cast to `timestamp`)
3. **Data Cleaning**
   - Standardize string columns (trim text).
   - Cast numeric fields (`Round`, `Year`) to integers.
   - Ensure consistent datetime format for start and end.
   - Drop duplicates.
4. **Write** results into Silver schema as `silver.session_info`.

## Output
- Clean and structured Silver table:
  `silver.session_info`


In [0]:
from pyspark.sql import functions as F

# Read from Bronze
session_info_bronze = spark.table("bronze.session_info")

# Transformations
session_info_silver = (
    session_info_bronze
    .select(
        F.col("Round").cast("int").alias("round"),
        F.col("Year").cast("int").alias("year"),
        F.trim(F.col("Circuit")).alias("circuit"),
        F.trim(F.col("SessionName")).alias("session_name"),
        F.trim(F.col("EventName")).alias("event_name"),
        F.to_timestamp("StartDate").alias("start_timestamp"),
        F.to_timestamp("EndDate").alias("end_timestamp")
    )
    .dropDuplicates()
)

# Write to Silver
(
    session_info_silver
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable("silver.session_info")
)
